In [1]:
import astropy.units as u
from astroduet.duet_telescope import load_telescope_parameters
from astroduet.duet_sensitivity import src_rate, bgd_sky_qe_rate, bgd_electronics, calc_exposure
from astroduet.bbmag import bb_abmag_fluence
import numpy as np
from matplotlib import pyplot as plt
from astroduet.bbmag import bb_abmag_fluence, bb_abmag

import astroduet.config as config
from astroduet.background import background_pixel_rate


%load_ext autoreload
%autoreload 2

from astropy.visualization import quantity_support
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

In [2]:

# Band1
# 5-sigma limiting magnitude in 3 stacked frames, CBE

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope()

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.01
print()

filter_type = 'B'
for qe_loss in [1.0]: #, 0.8, 0.77, 0.7, 0.5, 0.4]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 * qe_loss

    for nframes in [3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=12e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
        print('QE loss: {}'.format(qe_loss))
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
        print('Rates: ', src_rate, tot_bgd_rate)
        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()


duet1_cbe_limit = bbmag1



B
QE loss: 1.0
Band1 900.0 s 5-σ magnitude limit: 22.43368761357837 mag(AB)
Rates:  0.1396049959121671 ph / s 0.01753365098332382 ph / s
41.88149877365013 ph 5.260095294997146 ph



In [6]:

# Band2
# 5-sigma limiting magnitude in 3 stacked frames for 1.0, CBE

duet = config.Telescope()
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.01
print()
filter_type = 'B'
for qe_loss in [1.0]:#, 0.8, 0.77, 0.7, 0.5, 0.4]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band2 * qe_loss

    for nframes in [3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band2_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('QE loss: {}'.format(qe_loss))
        print('Band2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))
        print('Rates: ', src_rate, tot_bgd_rate)
        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()



duet2_cbe_limit = bbmag2



B
QE loss: 1.0
Band2 900.0 s 5-σ magnitude limit: 21.76198025121982 mag(AB)
Rates:  0.2429509268401836 ph / s 0.17922799624269026 ph / s
72.88527805205509 ph 53.768398872807076 ph



In [ ]:

# Band1
# 5-sigma limiting magnitude in 1 to 15 stacked frames for plotting

duet = config.Telescope()


#tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()

limit = []
frames = []
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1

    swiftmag = 20
    for nframes in np.arange(15):
        nframes += 1
        snr = 100
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
        print('Rates: ', src_rate, tot_bgd_rate)
        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()
        limit.append(bbmag1.value)
        frames.append(nframes)
        swiftmag -= 0.5


In [ ]:

# Band2
# 5-sigma limiting magnitude in 1 to 15 stacked frames for plotting

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope()


#tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()

limit2 = []
frames = []
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band2

    swiftmag = 20 


    for nframes in np.arange(15):
        nframes += 1
        snr = 100
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
        print('Band2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))
        print('Rates: ', src_rate, tot_bgd_rate)
        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()
        limit2.append(bbmag2.value)
        frames.append(nframes)
        swiftmag -= 0.5


In [ ]:
# Plot results
plt.scatter(frames, limit, label= 'DUET1')
plt.scatter(frames, limit2, label = 'DUET2')


plt.ylabel('5-sigma AB magnitude limit')
plt.xlabel('Number of Stacked Frames')
plt.legend()
plt.show()

f = open('DUET_limits.txt', 'w')
for fr, l1, l2, in zip(frames, limit, limit2):
    f.write('{} {} {}\n'.format(fr, l1, l2))
f.close()



In [20]:
def calc_margin(ab1, ab2):
    dmag = (ab1 - ab2)
    flux_ratio = 10**(0.4*dmag)
    margin = flux_ratio - 1.0
    return margin


def flux_ratio(ab1, ab2):
    dmag = (ab1 - ab2)
    flux_ratio = 10**(0.4*dmag)
    return flux_ratio


In [18]:
print(duet1_cbe_limit)
print(duet2_cbe_limit)



22.43368761357837 mag(AB)
21.76198025121982 mag(AB)


In [9]:
# Margin = 30% (average between the two) gives QE loss of 25%
duet = config.Telescope()
qe_loss1 = 0.70
#(area / area0) = (0.76) = (r**2 / r0**2)
# r**2 = r0**2 * 0.76
r = (0.5*duet.eff_epd) * np.sqrt(qe_loss1)
new_epd = 2*r
print(duet.EPD, new_epd)

# QE loss 2, 60% QE loss
qe_loss2 = 0.4
r = (0.5*duet.eff_epd) * np.sqrt(qe_loss2)
new_epd = 2*r
print(duet.EPD, new_epd)

26.0 cm 20.49817065008485 cm
26.0 cm 15.49516053482506 cm


In [10]:
# Go edit config.py with the above numbers for the eff_epd attribute
duet_cbe = config.Telescope()
duet_requirement = config.Telescope(config='minimum_mass_requirement')
duet_threshold = config.Telescope(config='minimum_mass_threshold')



In [11]:
# Confirmation taht works.
print(duet_cbe.eff_area, duet_requirement.eff_area, duet_requirement.eff_area / duet_cbe.eff_area)
print(duet_cbe.eff_area, duet_threshold.eff_area, duet_threshold.eff_area / duet_cbe.eff_area)



471.43524757931834 cm2 329.99917865997173 cm2 0.6999883448563099
471.43524757931834 cm2 188.5701916758644 cm2 0.39999171178675547


In [12]:
# Sanity check. Since we split the difference, actually getting a bit more margin for this one.
# Band1

# 5-sigma limit in three stacked frames, requirement
duet = config.Telescope(config='minimum_mass_requirement')

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.01
print()

filter_type = 'B'
for qe_loss in [1.0]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 * qe_loss

    for nframes in [3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=12e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
        print('QE loss: {}'.format(qe_loss))
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
        print('Rates: ', src_rate, tot_bgd_rate)
        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()


print('Margin: {}'.format(calc_margin(duet1_cbe_limit.value, bbmag1.value)))




B
QE loss: 1.0
Band1 900.0 s 5-σ magnitude limit: 22.08368761357832 mag(AB)
Rates:  0.1348937316862607 ph / s 0.012273351331105051 ph / s
40.468119505878214 ph 3.6820053993315156 ph

Margin: 0.38038426460294983


In [13]:

# Band2
# 5-sigma limit in three stacked frames, requirement


duet = config.Telescope(config='minimum_mass_requirement')
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.01
print()
filter_type = 'B'
for qe_loss in [1.0]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band2 * qe_loss

    for nframes in [3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band2_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('QE loss: {}'.format(qe_loss))
        print('Band2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))
        print('Rates: ', src_rate, tot_bgd_rate)
        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()


print('Margin: {}'.format(calc_margin(duet2_cbe_limit.value, bbmag2.value)))







B
QE loss: 1.0
Band2 900.0 s 5-σ magnitude limit: 21.51198025121978 mag(AB)
Rates:  0.21409640212424727 ph / s 0.1254575084418337 ph / s
64.22892063727419 ph 37.637252532550114 ph

Margin: 0.2589254117942166


In [21]:

# Band1
# 5-sigma limiting magnitude in 3 stacked frames for threshold

duet = config.Telescope(config='minimum_mass_threshold')

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.01
print()

filter_type = 'B'
for qe_loss in [1.0]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 * qe_loss

    for nframes in [3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=12e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
        print()



print('Margin: {}'.format(calc_margin(duet1_cbe_limit.value, bbmag1.value)))
print('Flux Ratio: {}'.format(flux_ratio(duet1_cbe_limit.value, bbmag1.value)))




B
Band1 900.0 s 5-σ magnitude limit: 21.513687613578227 mag(AB)

Margin: 1.333458062281312
Flux Ratio: 2.333458062281312


In [22]:

# Band2
# 5-sigma limiting magnitude in 3 stacked frames for threshold

duet = config.Telescope(config='minimum_mass_threshold')

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.01
print()

filter_type = 'B'
for qe_loss in [1.0]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band2 * qe_loss

    for nframes in [3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band2_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=12e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))
        print()



print('Margin: {}'.format(calc_margin(duet2_cbe_limit.value, bbmag2.value)))
print('Flux Ratio: {}'.format(flux_ratio(duet2_cbe_limit.value, bbmag2.value)))








B
Band1 900.0 s 5-σ magnitude limit: 20.739882231071306 mag(AB)

Margin: 1.563534745061999
Flux Ratio: 2.563534745061999


In [ ]:

# Band2
# 5-sigma limiting magnitude in 1 and 5 stacked frames for requirement

duet = config.Telescope(config='minimum_mass_requirement')
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.01
print()
filter_type = 'B'
for qe_loss in [1.0]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band2 * qe_loss

    for nframes in [3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band2_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('QE loss: {}'.format(qe_loss))
        print('Band2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))
        print('Rates: ', src_rate, tot_bgd_rate)
        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()

print('Margin: {}'.format(calc_margin(duet2_cbe_limit.value, bbmag2.value)))







In [4]:

# Band1
# 5-sigma limiting magnitude in 1, 3, and 5 stacked 60-s frames, for requirement

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope(config='minimum_mass_requirement')

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 60*u.s
print()
siglimit=5
dmag = 0.01
print()

filter_type = 'B'
for qe_loss in [1.0]: #, 0.8, 0.77, 0.7, 0.5, 0.4]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True, filter_type=filter_type, diag=True)
    tot_bgd_rate = bgd_band1 * qe_loss

    for nframes in [1, 5, 15]:
        snr = 100
        swiftmag = 19
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=12e3*u.K, bandone = bandone, bandtwo=bandtwo) 

#        print('SNR: {}'.format(snr))
#        print('QE loss: {}'.format(qe_loss))
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
#        print('Rates: ', src_rate, tot_bgd_rate)
#        print('Inputs:', src_rate*exposure*nframes, tot_bgd_rate*exposure*nframes*duet.neff, nframes*duet.neff*np.sqrt(duet.read_noise))
        
    print()





B
-----
Background Computation Integrating over Pixel Area
Telescope diameter: 26.0 cm


Pixel Size: 6.559999999999999 arcsec
Pixel Area: 43.033599999999986 arcsec2

Zodi Level: 900
Band1 Rate: 0.03535372451555378 ph / s
Band2 Rate: 1.0923879988913832 ph / s
-----
Band1 60.0 s 5-σ magnitude limit: 19.6836876135781 mag(AB)
Band1 300.0 s 5-σ magnitude limit: 20.673687613578252 mag(AB)
Band1 900.0 s 5-σ magnitude limit: 21.313687613578352 mag(AB)



In [6]:

# Band2
# 5-sigma limiting magnitude in 1, 3, and 5 stacked 60-s frames

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope(config='minimum_mass_requirement')

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 60*u.s
print()
siglimit=5
dmag = 0.01
print()

filter_type = 'B'
for qe_loss in [1.0]: #, 0.8, 0.77, 0.7, 0.5, 0.4]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band2 * qe_loss

    for nframes in [1, 5, 15]:
        snr = 100
        swiftmag = 18
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band2_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=12e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
#        print('QE loss: {}'.format(qe_loss))
        print('Band2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))
#        print('Rates: ', src_rate, tot_bgd_rate)
#        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()






B
Band2 60.0 s 5-σ magnitude limit: 18.569882231071276 mag(AB)

Band2 300.0 s 5-σ magnitude limit: 19.499882231071425 mag(AB)

Band2 900.0 s 5-σ magnitude limit: 20.11988223107152 mag(AB)



In [ ]:

# 75-frames stacking (1-day)

duet = config.Telescope()

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.5
print()

filter_type = 'B'
for qe_loss in [1.0]: #, 0.8, 0.77, 0.7, 0.5, 0.4]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 * qe_loss

    for nframes in [75]:
        snr = 100
        swiftmag = 22
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=12e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
#        print('QE loss: {}'.format(qe_loss))
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
#        print('Rates: ', src_rate, tot_bgd_rate)
#        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()


duet1_cbe_limit = bbmag1